D2D (존편도/부름편도 / 부름)는 이익 증분에 도움을 주는가 ?

1. 부름 예약은 실제로 사업 성장(매출 증대, 이익 개선)에 도움을 줄 수 있는가 ?
* 제주 존 왕복 vs 부름 왕복
* 내륙 존 왕복 vs 부름 왕복 VS 부름 편도 VS 존편도
* 내륙(부산, 호남) 존 왕복 VS 부름 왕복 VS 부름 편도 VS 존편도

2. 제주에 존 편도 / 부름 편도가 오픈된다면 매출 순증분액과 이익 증분액에 도움을 줄 수 있을 것인가
* 제주 예상 건수와 매출

3. 결론
* 존 편도와 부름 편도를 오픈해야하는가 ?
* 목표는 어느정도가 적정한가
* 성과 측정 지표 수립 (왕복 건수 유지/운행 타입별 건수 전체 구성비, 순매출 증가액, 순이익 증가액, 핸들 운행률)

---



# 부름 예약 EDA
개요: D2D (존편도/부름편도 / 부름)는 이익 증분에 도움을 주는가 ?
문제: 코로나이후 23년부터 제주 여행의 정체기로 매출이 감소되지만 목표 매출은 증가되었음
해결: 이용의 허들을 낮춰, 이용을 개선하고 매출을 신장시킨다
```
부름서비스의 지역을 확대하고 부름 편도 서비스를 오픈한다
```
1. 검증1 전국 지역별 예약 구성비 비교
: 대략적인 예약 구성비를 비교해보고 비슷한 지역의 구성비를 참조하여 제주 구성비를 예샹
```
대체로 왕복 예약의 비중이 높음
```

2. 검증2 부름서비스는 매출 신장에 기여할 수 있는가
: 수도권과 비슷한 특성의 지역의 건당 실적 지표를 비교하여 제주 실적을 예상
```
부름서비스의 실적 지표는 모든 면에서 왕복대비 크게 높음
```
결론
1. 부름서비스의 지역 확대와 편도 서비스 오픈은 제주 매출 신장에 크게 기여할 수 있다
2. 가동률과 유휴율도 개선할 수 있을 것으로 기대

명세서
예약타입 | 예약의 운행타입 | 쏘카존 왕복, 부름 왕복, 부름 편도, 존 편도로 나누어짐
총매출 | 전체 매출 |
공헌이익 | 변동비가 빠진 고정비 원가 회수 개념 |
건당 총매출 | 예약 건당 총매출 |
건당 공헌이익 | 예약 건당 공헌이익 |

* 모든 수치는 정규화로 표현 *


In [ ]:
from google.colab import output
output.clear()

# 필요한 라이브러리 준비 ⏳
---

In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import plotly.express as px

import gspread
from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.cloud import bigquery
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(creds)

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년 1월 ~11월

In [ ]:
query = """
  SELECT
      extract(year from date) as year
      , extract(month from date) as month
      , r.isoweek as isoweek
      , CASE WHEN  extract(month FROM date) IN (1, 2, 3) THEN '1Q'
            WHEN  extract(month FROM date) IN (4, 5, 6) THEN '2Q'
            WHEN  extract(month FROM date) IN (7, 8, 9) THEN '3Q' ELSE '4Q' END as q
      , r.way as way
      , case when r.way IN ("d2d_round", "d2d_oneway") then 'd2d'
            when r.way = "z2d_oneway" then "z2d_oneway" else 'round' end as way_g

      , r.region1
      , r.region2

      , cl.car_model

      , count(r.reservation_id) as nuse -- 예약건수
      , sum(r.utime) as utime -- 이용시간

      , sum(revenue) as revenue -- 회계매출
      , sum(_rev_d2d) as _rev_d2d -- 부름매출(부름요금 + 편도요금)

      , sum(contribution_margin) as margin -- 공헌이익

  FROM `socar-data.soda_store.reservation_v2` r
  LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
  LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
  LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
  LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
  WHERE 1=1
  AND date BETWEEN '2023-01-01' AND '2023-11-30'
  AND r.member_imaginary IN (0,9)
  GROUP BY isoweek, date, way, way_g, month, region1, region2, car_model
  ORDER BY isoweek, date, way, way_g, month, region1, region2, car_model
  """

df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
pd.set_option('display.float_format', '{:1.0f}'.format)

In [ ]:
df.dtypes

In [ ]:
df.describe()

# Part 1_ 1. 부름 예약은 실제로 사업 성장(매출 증대, 이익 개선)에 도움을 줄 수 있는가 ?

* 지역별로 예약 타입에 따른 건수의 구성비는 어떤가

## 📈 *지역별 건수와 예약타입별 구성비*
- 모든 지역을 보니 보기가 어렵다
- 주요 도시와 제주와 비슷한 지역(호남, 부산)만을 따로 본다

✔ 대부분의 지역에서 왕복 > 부름 왕복 > 존편도 > 부름편도로 건수 비중이 높다
- 제주는 부름 편도와 존편도가 오픈되어 있지 않기 때문에 왕복 비중이 더 높게 나타남

In [ ]:
# region1과 way_g별로 nuse의 합계를 계산
summary = df.groupby(['region1', 'way'])['nuse'].sum().reset_index()

# region1별로 nuse의 총합을 계산
region_totals = df.groupby(['region1'])['nuse'].sum().reset_index().rename(columns={'nuse': 'total_nuse'})

# summary에 region1별 총합을 merge하여 추가
summary = summary.merge(region_totals, on='region1')

# 각 way_g의 nuse가 region1 내에서 차지하는 비율을 계산
summary['proportion'] = (summary['nuse'] / summary['total_nuse']) * 100

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='proportion', color='region1',
             title='지역 내 예약타입별 nuse의 구성비',
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
df.loc[df['region1'].isin(['전라남도', '전라북도']), 'region1'] = '호남'

# region1이 특정 지역들인 행만 선택
df_part = df[(df['region1'] == '제주특별자치도') | (df['region1'] == '서울특별시')  | (df['region1'] == '부산광역시')  | (df['region1'] == '호남')   | (df['region1'] == '부산광역시')  | (df['region1'] == '경기도')]

# region1과 way_g별로 nuse의 합계를 계산
summary = df_part.groupby(['region1', 'way'])['nuse'].sum().reset_index()

# region1별로 nuse의 총합을 계산
region_totals = df_part.groupby(['region1'])['nuse'].sum().reset_index().rename(columns={'nuse': 'total_nuse'})

# summary에 region1별 총합을 merge하여 추가
summary = summary.merge(region_totals, on='region1')

# 각 way_g의 nuse가 region1 내에서 차지하는 비율을 계산
summary['proportion'] = (summary['nuse'] / summary['total_nuse']) * 100

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='proportion', color='region1',
             title='지역 내 예약타입별 nuse의 구성비',
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

## 📈 *지역별 매출과 매출 대비 공헌이익도 살펴보기*
1. 매출 구성비는 건수 구성비를 따른다 (당연하게도 건수가 많다면 매출이 높다)
2. 공헌이익률은 건수와 상관 없이 대체로 비슷하다

In [ ]:
# region1이 특정 지역들인 행만 선택
df_part = df[(df['region1'] == '제주특별자치도') | (df['region1'] == '서울특별시')  | (df['region1'] == '부산광역시')  | (df['region1'] == '호남')   | (df['region1'] == '부산광역시')  | (df['region1'] == '경기도')]

# region1과 way_g별로 nuse의 합계를 계산
summary = df_part.groupby(['region1', 'way'])['revenue'].sum().reset_index()

# region1별로 nuse의 총합을 계산
region_totals = df_part.groupby(['region1'])['revenue'].sum().reset_index().rename(columns={'revenue': 'total_revenue'})

# summary에 region1별 총합을 merge하여 추가
summary = summary.merge(region_totals, on='region1')

# 각 way_g의 nuse가 region1 내에서 차지하는 비율을 계산
summary['proportion'] = (summary['revenue'] / summary['total_revenue']) * 100

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='proportion', color='region1',
             title='지역 내 예약타입별 revenue 구성비',
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
# 단계 1: region1과 way_g별로 margin과 revenue의 합계를 계산
summary = df_part.groupby(['region1', 'way']).agg({'margin': 'sum', 'revenue': 'sum'}).reset_index()

# 단계 2: 각 그룹별 공헌이익률(margin_ratio) 계산
summary['margin_ratio'] = (summary['margin'] / summary['revenue']) * 100

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='margin_ratio', color='region1',
             title='지역 내 예약타입별 공헌이익률',
             barmode='group',
             text='margin_ratio',
             labels={'margin_ratio': '공헌이익률 (%)', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.show()

## 📈 *예약타입별 건당 매출과 건당 공헌이익 살펴보기*
1. 각 건의 매출과 공헌이익을 살펴보았을 때   
   부름서비스(편도/왕복)의 매출과 공헌이익이 왕복 예약 대비 매우 높았으며,
3. 제주와 비슷한 운영지역(부산/경기도)에서는 왕복 예약 보다 존 편도의 건당 매출과 공헌이익이 높다

✔ 부름 편도와 존 편도 건은 왕복 대비 매출과 공헌이익이 우수하다 -> 오픈시 매출과 공헌이익의 개선을 기대할 수 있다

In [ ]:
# 단계 1: region1과 way 별로 revenue와 nuse의 합계를 계산
summary = df_part.groupby(['region1', 'way']).agg({'revenue': 'sum', 'nuse': 'sum'}).reset_index()

# 단계 2: 각 그룹별 건당 매출(revenue_nuse) 계산
summary['revenue_nuse'] = summary['revenue'] / summary['nuse']

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='revenue_nuse', color='region1',
             title='지역 내 예약타입별 건당 매출',
             barmode='group',
             text='revenue_nuse',
             labels={'revenue_nuse': '건당 매출', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 단계 1: region1과 way 별로 revenue와 nuse의 합계를 계산
summary = df_part.groupby(['region1', 'way']).agg({'revenue': 'sum', 'nuse': 'sum'}).reset_index()

# 단계 2: 각 그룹별 건당 매출(revenue_nuse) 계산
summary['revenue_nuse'] = summary['revenue'] / summary['nuse']

# 정규화 진행
scaler = MinMaxScaler()
summary['revenue_nuse_normalized'] = scaler.fit_transform(summary[['revenue_nuse']])

# 바 차트로 정규화된 결과 시각화
fig = px.bar(summary, x='way', y='revenue_nuse_normalized', color='region1',
             title='지역 내 예약타입별 정규화된 건당 매출',
             barmode='group',
             text='revenue_nuse_normalized',
             labels={'revenue_nuse_normalized': '정규화된 건당 매출', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # 소수점 두 자리로 표시
fig.show()


In [ ]:
# 단계 1: region1과 way 별로 revenue와 nuse의 합계를 계산
summary = df_part.groupby(['region1', 'way']).agg({'margin': 'sum', 'nuse': 'sum'}).reset_index()

# 단계 2: 각 그룹별 건당 매출(revenue_nuse) 계산
summary['margin_nuse'] = summary['margin'] / summary['nuse']

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='margin_nuse', color='region1',
             title='지역 내 예약타입별 건당 공헌이익',
             barmode='group',
             text='margin_nuse',
             labels={'margin_nuse': '건당 공헌이익', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 단계 1: region1과 way 별로 margin과 nuse의 합계를 계산
summary = df_part.groupby(['region1', 'way']).agg({'margin': 'sum', 'nuse': 'sum'}).reset_index()

# 단계 2: 각 그룹별 건당 공헌이익(margin_nuse) 계산
summary['margin_nuse'] = summary['margin'] / summary['nuse']

# 정규화 진행
scaler = MinMaxScaler()
summary['margin_nuse_normalized'] = scaler.fit_transform(summary[['margin_nuse']])

# 바 차트로 정규화된 결과 시각화
fig = px.bar(summary, x='way', y='margin_nuse_normalized', color='region1',
             title='지역 내 예약타입별 정규화된 건당 공헌이익',
             barmode='group',
             text='margin_nuse_normalized',
             labels={'margin_nuse_normalized': '정규화된 건당 공헌이익', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # 소수점 두 자리로 표시
fig.show()

## 📈 *예약타입별 건당 이용시간 살펴보기*
1. 건당 이용시간 역시 부름 예약이 왕복대비 압도적으로 높다
** 제주는 운영중인 서비스가 아니나, 고객센터에서 임의로 대차되어 생성된 몇몇 건들로 서비스중인 것처럼 나타나니 무시한다 **

In [ ]:
# "제주특별자치도"의 "d2d_oneway" 데이터 제외 (고객센터에서 임의로 예악타입을 변경한 케이스)
filtered_df = df_part[~((df_part['region1'] == '제주특별자치도') & (df_part['way'] == 'd2d_oneway'))]

# 단계 1: region1과 way 별로 revenue와 nuse의 합계를 계산
summary = filtered_df.groupby(['region1', 'way']).agg({'utime': 'sum', 'nuse': 'sum'}).reset_index()

# 단계 2: 각 그룹별 건당 매출(revenue_nuse) 계산
summary['dur_nuse'] = summary['utime'] / summary['nuse']

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='dur_nuse', color='region1',
             title='지역 내 예약타입별 건당 이용시간',
             barmode='group',
             text='dur_nuse',
             labels={'dur_nuse': '건당 이용시간', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# "제주특별자치도"의 "d2d_oneway" 데이터 제외 (고객센터에서 임의로 예악타입을 변경한 케이스)
filtered_df = df_part[~((df_part['region1'] == '제주특별자치도') & (df_part['way'] == 'd2d_oneway'))]

# 단계 1: region1과 way 별로 utime과 nuse의 합계를 계산
summary = filtered_df.groupby(['region1', 'way']).agg({'utime': 'sum', 'nuse': 'sum'}).reset_index()

# 단계 2: 각 그룹별 건당 이용시간(dur_nuse) 계산
summary['dur_nuse'] = summary['utime'] / summary['nuse']

# 정규화 진행
scaler = MinMaxScaler()
summary['dur_nuse_normalized'] = scaler.fit_transform(summary[['dur_nuse']])

# 바 차트로 정규화된 결과 시각화
fig = px.bar(summary, x='way', y='dur_nuse_normalized', color='region1',
             title='지역 내 예약타입별 정규화된 건당 이용시간',
             barmode='group',
             text='dur_nuse_normalized',
             labels={'dur_nuse_normalized': '정규화된 건당 이용시간', 'way': '예약타입', 'region1': '운영지역시/도'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # 소수점 두 자리로 표시
fig.show()

# 결론
---

1. 부름 편도와 존 편도의 오픈은 전반적인 매출 개선과 공헌이익 개선에 기여할 수 있음
2. 제주의 경우, 현재 부름 운영을 한정적인 지역과 차량으로 운영하고 있어 이를 확대한다면 건수와 함께 매출 개선을 기대할 수 있을 것으로 보여짐
- 23y 11월 기준 제주 부름 운영지역 (제주시내 일부, 서귀포 시내 일부에 대해 제주공항 일부 차량 및 서귀포 일부 차량 이용 가능)